# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("create keyspace if not exists music_events with REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## model the data to be able to retrieve the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [8]:
try:
    session.execute("drop table if exists sessions")
except Exception as e:
    print(e)

query = "create table if not exists sessions"
query = query + "(session_id int, item_insession int, song_title text, artists text, length float, primary key (session_id,item_insession ))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# inserting data from csv file to cassandra (table: sessions)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into sessions (session_id, item_insession, song_title, artists, length)"
        query = query + "values(%s,%s,%s,%s,%s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), str(line[9]), str(line[0]), float(line[5])))

#### SELECT statement to verify the data was entered into the table sessions

In [10]:
query = ("select artists, song_title, length from sessions where session_id = 338 and item_insession = 4")
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artists, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### model the data to be able to retrieve: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182

In [11]:
try:
    session.execute("drop table if exists user_activities")
except Exception as e:
    print(e)

query = "create table if not exists user_activities"
query = query + "(user_id int, session_id int, song_title text, artist text, f_name text, l_name text, item_insession int,  primary key ((user_id,session_id), item_insession ))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# inserting data from csv file to cassandra (table: user_activities)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_activities (user_id, session_id, song_title, artist, f_name, l_name, item_insession)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), str(line[9]), str(line[0]), str(line[1]), str(line[4]), int(line[3])))

In [ ]:
#### SELECT statement to verify the data was entered into the table user_activities

In [21]:
query = "select artist, song_title, f_name, l_name from user_activities where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.f_name, row.l_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### model the data to be able to retrieve: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
try:
    session.execute("drop table if exists who_listen")
except Exception as e:
    print(e)

query = "create table if not exists who_listen"
query = query + "(user_id int,  song_title text, f_name text, l_name text, primary key (song_title ,user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
# inserting data from csv file to cassandra (table: who_listen)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into who_listen (user_id, song_title, f_name, l_name)"
        query = query + "values(%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), str(line[9]), str(line[1]), str(line[4])))

In [20]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select  f_name, l_name from who_listen where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.f_name, row.l_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
try:
    session.execute("drop table if exists sessions")
except Exception as e:
    print(e)

try:
    session.execute("drop table if exists user_activities")
except Exception as e:
    print(e)
    
try:
    session.execute("drop table if exists who_listen")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()